In [ ]:
%matplotlib inline
import pandas as pd
import os
import networkx as nx
import igraph as ig
import numpy as np
import louvain
import matplotlib.pyplot as plt

In [ ]:
filepath = '/media/sf_VBox_Shared/CaseLaw/2018-01-29-lido/derived/'
links_df = pd.read_csv(os.path.join(filepath, 'case_to_article_title_links.csv'))

In [ ]:
case_to_case_links = pd.read_csv(os.path.join(filepath, 'case_to_case_lx_links.csv'))

In [ ]:
art_nodes = pd.read_csv(os.path.join(filepath, 'article_title_nodes.csv'))
case_nodes = pd.read_csv(os.path.join(filepath, 'case_nodes_simple.csv'))

In [ ]:
case_nodes['id'] = case_nodes['lido_id']
case_nodes['label'] = case_nodes['ecli'].str.split(':').map(lambda l: '{}:{}'.format(l[2], l[3]) if type(l)==list else l)

In [ ]:
art_nodes.head()

In [ ]:
case_nodes.head()

In [ ]:
seed_articles = art_nodes[art_nodes.label.str.match('Burgerlijk Wetboek Boek 7, Artikel ((611)|(658))', case=False)]
seed_articles

In [ ]:
linking_cases = links_df[links_df.target.isin(seed_articles.id)]['source'].unique()
len(linking_cases)

In [ ]:
links_sub = links_df[links_df.source.isin(linking_cases)]
links_sub.shape

In [ ]:
case_to_case_sub = case_to_case_links[case_to_case_links.source.isin(linking_cases)&case_to_case_links.target.isin(linking_cases)]
case_to_case_sub.shape

In [ ]:
art_nodes_sub = art_nodes[art_nodes.id.isin(links_sub.target.unique())]
case_nodes_sub = case_nodes[case_nodes.lido_id.isin(linking_cases)]
print(art_nodes_sub.shape, case_nodes_sub.shape)

In [ ]:
case_to_case_sub.to_csv(os.path.join(filepath, 'subnetwork', 'case-to-case-links.csv'), index=False)
links_sub.to_csv(os.path.join(filepath, 'subnetwork', 'case-to-article-links.csv'), index=False)
art_nodes_sub.to_csv(os.path.join(filepath, 'subnetwork', 'article-nodes.csv'), index=False)
case_nodes_sub.to_csv(os.path.join(filepath, 'subnetwork', 'case-nodes-sub.csv'), index=False)

## enrich
Use caselawnet for these nodes

In [ ]:
import caselawnet

In [ ]:
links_dict = [{'source': d['source'].split('/')[-1], 'target': d['target'].split('/')[-1]} for d in case_to_case_sub.to_dict(orient='records')]

In [ ]:
enrich_again = False

if enrich_again:
    nodes_rich_list = caselawnet.enrich_eclis([s.split('/')[-1] for s in case_nodes_sub.id ], rootpath='/media/sf_VBox_Shared/CaseLaw/OpenDataUitspraken/')
    nodes_rich = pd.DataFrame(nodes_rich_list)
    nodes_rich.to_csv(os.path.join(filepath, 'subnetwork', 'nodes-cases-rich.csv'), index=False)
else:
    nodes_rich = pd.read_csv(os.path.join(filepath, 'subnetwork', 'nodes-cases-rich.csv')).fillna('')
    nodes_rich_list = nodes_rich.to_dict(orient='records')

In [ ]:
nodes_rich.head()

In [ ]:
links_list = caselawnet.enrich_links(links_dict)

In [ ]:
nodes_network, links_list = caselawnet.get_network(nodes_rich_list, links_list)

In [ ]:
caselawnet.to_sigma_json(nodes_network, links_list, 'Employer liability', os.path.join(filepath, 'subnetwork', 'network.json'))

In [ ]:
nodes_network_df = pd.DataFrame(nodes_network)
nodes_network_df.community.value_counts().head()

In [ ]:
graph = caselawnet.network_analysis.get_network(nodes_rich_list, links_list)

In [ ]:
len(graph)

In [ ]:
graph = nx.readwrite.json_graph.node_link_graph({'nodes': nodes_rich_list, 'links': links_list},
                                       directed=True, multigraph=False)

In [ ]:
graph.number_of_nodes(), graph.number_of_edges()

In [ ]:
def add_network_statistics(nodes, links):
    if len(nodes)==0:
        return nodes
    graph = get_network(nodes, links)
    degree = nx.degree(graph)
    if max(dict(degree).values()) > 0:
        hubs, authorities = get_hits(graph)
        statistics = {
            'degree': degree,
            'in_degree': graph.in_degree(),
            'out_degree': graph.out_degree(),

            'degree_centrality': nx.degree_centrality(graph),
            'in_degree_centrality': nx.in_degree_centrality(graph),
            'out_degree_centrality': nx.out_degree_centrality(graph),
            'betweenness_centrality': nx.betweenness_centrality(graph),
            'closeness_centrality': nx.closeness_centrality(graph),
            'pagerank': get_pagerank(graph),
            'hubs': hubs,
            'authorities': authorities
        }
    else:
        statistics = {}

    # for relative in-degree we sort on date
    derive_date = lambda k: k['date'] if k['date']!='' else '{}-01-01'.format(k['year'])
    nodes.sort(key=derive_date, reverse=True)
    for i, node in enumerate(nodes):
        nodeid = node['id']
        for var in statistics.keys():
            node[var] = statistics[var][nodeid]
        if 'in_degree' in node:
            node['rel_in_degree'] = node['in_degree'] / float(max(i, 1))
    get_community(graph, nodes)
    return nodes

In [ ]:
case_nodes_sub.court.value_counts()

In [ ]:
art_nodes_sub.book.value_counts()